In [1]:
import ast
import boto3
import json
import math
import numpy as np
import os
import pandas as pd
import requests
import time
import torch
import torch.nn.functional as F

from botocore.exceptions import ClientError
from decimal import Decimal
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
#from torch.nn.functional import cosine_similarity

In [2]:
# uploading the environment variables and get the API key
load_dotenv()
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")

### User Preferences from DynamoDB

In [13]:
# To get the info from DynamoDB, user preferences
CONFIG = {
    'aws': {
        'access_key': AWS_ACCESS_KEY,
        'secret_key': AWS_SECRET_KEY,
        'region': 'eu-west-3',
        'table': 'User-7kkcm5dn2rb77hst5nh7gbdisa-staging'
    },
    'columns': ['userId', 'favoriteMoviesIds', 'favoriteGenresIds', 'favoriteSeriesIds'],
}

# conexion
session = boto3.Session(
    aws_access_key_id=CONFIG['aws']['access_key'],
    aws_secret_access_key=CONFIG['aws']['secret_key'],
    region_name=CONFIG['aws']['region']
)

table = session.resource('dynamodb').Table(CONFIG['aws']['table'])

# Values to String
def _process_value(value):
    if isinstance(value, Decimal):
        return str(int(value))
    return str(value)

# Retrive info from DynamoDB and gets a DataFrame
def fetch_preferences():
    try:
        items = []
        start_key = None

        while True:
            # scan with defined 'columns'  in previous 'CONFIG'
            scan_params = {
                'ProjectionExpression': ', '.join(CONFIG['columns'])
            }
            if start_key:
                scan_params['ExclusiveStartKey'] = start_key

            response = table.scan(**scan_params)
            items.extend(response.get('Items', []))

            # check for next pages
            start_key = response.get('LastEvaluatedKey')
            if not start_key:
                break

        # data extracted processing
        processed_data = [{
            'userId': _process_value(item.get('userId', '')),
            'favoriteMoviesIds': ';'.join(map(_process_value, item.get('favoriteMoviesIds', []))),
            'favoriteGenresIds': ';'.join(map(_process_value, item.get('favoriteGenresIds', []))),
            'favoriteSeriesIds': ';'.join(map(_process_value, item.get('favoriteSeriesIds', [])))
        } for item in items]

        df = pd.DataFrame(processed_data)
        return df

    except ClientError as e:
        print(f"Error al conectar con DynamoDB: {e}")
        return pd.DataFrame()

# calling function to get the df
user_pref = fetch_preferences()


In [ ]:
# limpio el dataframe dejando solo users con genero, movie_favs y tvshow_favs
user_pref = user_pref[user_pref['userId'].str.len()==36]
user_pref = user_pref.replace("",pd.NA)
user_pref = user_pref.dropna()
user_pref.reset_index(inplace=True,drop=True)
print(f'Duplicates: {user_pref.duplicated().sum()}')
user_pref.info()

Duplicates: 0
<class 'pandas.core.frame.DataFrame'>
Index: 5133 entries, 0 to 13665
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   userId             5133 non-null   object
 1   favoriteMoviesIds  5133 non-null   object
 2   favoriteGenresIds  5133 non-null   object
 3   favoriteSeriesIds  5133 non-null   object
dtypes: object(4)
memory usage: 200.5+ KB


In [5]:
# en esta parte el codigo queda freezado mientras se trabaja en el contenido a obtener desde fire
# ya que necesito ir a buscar los 'Genres', 'CleanTitle', y 'Synopsis' en la base de Fire Base para crear las sentences

In [42]:
# esta celda es temporal por lo explicado en la celda anterior
# importo los generos desde archivo local
genres = pd.read_csv('generos.csv', sep=',')
genres.columns

Index(['genero_id', 'genero_name'], dtype='object')

In [49]:
# esta celda es temporal por lo explicado en la celda anterior
# traigo generos a la base para que quede lista para las sentences (y luego embeddings)

df_prueba = user_pref.head(100)


# diccionario (matriz destino) para hacer una onda vlookup
genre_dict = genres.set_index('genero_id')['genero_name'].to_dict()

# la vlookup 
def map_genres(genre_ids):
    if pd.isna(genre_ids) or not genre_ids.strip():  # nan o null?
        return 'Unknown'
    try:
        # genre_ids que sea str
        return ';'.join(genre_dict.get(int(g), 'Unknown') for g in genre_ids.split(';'))
    except Exception as e:
        return 'Unknown'

# aplico la funcion
user_pref['Genre_Text'] = user_pref['favoriteGenresIds'].apply(map_genres)
user_pref


,userId,favoriteMoviesIds,favoriteGenresIds,favoriteSeriesIds,Genre_Text
0,c1f9d00e-6001-70e0-28b6-9fcaee14754c,24428;299536;118340;150540;283995,16;10762;14;10765;878;12;10759;10751,66732;93405;456;119051;65334,Animación;Kids;Fantasía;Sci-Fi & Fantasy;Cienc...
1,f139102e-50c1-70e7-9ccb-063a02d6fe53,157336;155;293660;24428;299536,28;10759;12;99;10764;10763;80;18;27;14;10752;1...,1399;66732;71712;84958;60735,Acción;Action & Adventure;Aventura;Documental;...
2,1199a0fe-d011-709f-c346-f753a0418008,13;278;603;475557;597,35;10749;28;10759;16;10762;12;10751;18;10402;9...,1399;71446;66732;1396;69050,Comedia;Romance;Acción;Action & Adventure;Anim...
3,c159c01e-e081-7026-6d61-77d3d6ce81fd,27205;157336;155;19995;293660,28;10759;27;878;10765;35;12,456;60625;75006;18165;119051,Acción;Action & Adventure;Terror;Ciencia ficci...
4,3199f02e-6081-7073-51ce-20f5ad08ea50,27205;157336;155;19995;293660,35;28;10759;18;12;80;16;10762;99;10764;10763;1...,1399;71446;66732;1402;93405;112888,Comedia;Acción;Action & Adventure;Drama;Aventu...
...,...,...,...,...,...
5128,e149f0de-4031-70aa-b8ae-f5e9efe5bdb4,155;671;603;11324;106646;1124;45612;2832;322,28;10759;12;10770;10766;10767;18;10751;16;1076...,71446;85552;18165;70523;87108,Acción;Action & Adventure;Aventura;Película de...
5129,5129108e-0001-7078-eb08-e2fcfdc6c8d1,27205;157336;155;19995;24428,10766;10767;10770;80;10752;10768;53;878;10765;...,1399;71446;66732;1402;63174,Soap;Talk;Película de TV;Crimen;Bélica;War & P...
5130,c1a960be-d0c1-70b4-f203-4e1efa0ee3a1,27205;13;278;68718;597;769,10749;18;35;10751;36;14;10765;12;10759,1399;71446;1396;1418;60574,Romance;Drama;Comedia;Familia;Historia;Fantasí...
5131,0159703e-1001-7005-e57c-a2ad2e33e530,671;278;1726;68718;603,10770;10766;10767;37;16;10762;99;10764;10763;1...,1399;66732;1402;63174;1396,Película de TV;Soap;Talk;Western;Animación;Kid...


In [48]:
print(df_prueba.Genre_Text.value_counts())
print(df_prueba.isna().sum())

Genre_Text
Unknown    27
Name: count, dtype: int64
userId                0
favoriteMoviesIds     0
favoriteGenresIds     0
favoriteSeriesIds     0
Genre_Text           73
dtype: int64


### Content Data from FireBase

In [ ]:
import requests
import json

# URL de la base de datos
url = "https://bubbo-dfba0-default-rtdb.europe-west1.firebasedatabase.app/documents/Content_latest_ad_flixole_jsonl.json"

# Realizar la solicitud GET
response = requests.get(url)

if response.status_code == 200:
    # Convertir la respuesta en un diccionario
    data = response.json()

    # Obtener las primeras 10 claves
    first_10 = list(data.items())[:10]  # Extrae los primeros 10 elementos como lista de pares clave-valor

    # Mostrar los resultados
    print("Primeros 10 registros:")
    for key, value in first_10:
        print(f"Clave: {key}, Valor: {value}")
else:
    print(f"Error al acceder a la base de datos: {response.status_code}")


In [20]:
# From FireBase
# filtered_data = pd.read_csv(r'../../../filtered_data.csv',sep=';')

### Sentences for vectorize from 'filtered_data' / AHORA DESDE FIREBASE

In [ ]:
# Making the sentences to embed 
 

# Formatting as str-list to send to the model
sentences_from_filtered_data = filtered_data.sentences_to_embed.dropna().astype(str).tolist()


In [7]:
# Since the model on Hugging Face processes only requests that can be completed within 60 seconds, we need to divide the sentences into batches.
def split_into_batches(sentences, batch_size):
    return [sentences[i:i + batch_size] for i in range(0, len(sentences), batch_size)]

# After trying with different values, we've reach the maximum batch size to get response succesfully
batches = split_into_batches(sentences_from_filtered_data, 100)

In [ ]:
# Check key availability
if HUGGINGFACE_API_KEY is None:
    print("Error: No se encontró la clave de API de Hugging Face.")
else:
    print("Clave de API cargada correctamente.")

# Model URL
API_URL = "https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/all-MiniLM-L6-v2"

# API header and key
headers = {"Authorization": f"Bearer {HUGGINGFACE_API_KEY}"}  

# Function to get embeddings from Hugging Face API
def get_embeddings_from_api(sentences):
    url = API_URL
    payload = {"inputs": sentences}
    
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None
    

# Sequence Batch Processing Process
all_embeddings_from_filtered_data = []
for batch in batches:
    print(f"Processing batch with {len(batch)} sentences...")
    embeddings = get_embeddings_from_api(batch)
    if embeddings:
        all_embeddings_from_filtered_data.extend(embeddings)

print("Embeddings processed successfully:")
print(all_embeddings_from_filtered_data[:2])  

### Sentences for vectorize from 'user_preferences' / AHORA DESDE DYNAMODB

In [9]:
# Sentences we want to be embedded from user_preferences
user_pref['sentences_to_embed'] = (user_pref.TitulosPeliculas.fillna('') +
                                   user_pref.TitulosSeries.fillna('') + 
                                   user_pref.GenerosFavoritos.fillna('') +
                                   user_pref.DetallesPeliculas.fillna('') +
                                   user_pref.DetallesSeries.fillna(''))

# Formatting as str-list to send to the model
sentences_from_user_pref = user_pref.sentences_to_embed.dropna().astype(str).tolist()

# We split the sentences in batches as we did previously with filtered_data
batches_user_pref = split_into_batches(sentences_from_user_pref, 100)

In [ ]:
# Sequence Batch Processing Process
all_embeddings_from_user_pref = []
for batch in batches_user_pref:
    print(f"Processing batch with {len(batch)} sentences...")
    embeddings = get_embeddings_from_api(batch)
    if embeddings:
        all_embeddings_from_user_pref.extend(embeddings)

print("Embeddings processed successfully:")
print(all_embeddings_from_user_pref[:2])  

### Similarities

In [24]:
# We have to convert the embeddings list to numpy arrays in order to calculate cosine similarities wiht sklearn
all_embeddings_from_user_pref_array = np.array(all_embeddings_from_user_pref)
all_embeddings_from_filtered_data_array = np.array(all_embeddings_from_filtered_data)

user_for_example = 1

# Taking first user as example to calculate the cosine_similarity
user_embedding_example = all_embeddings_from_user_pref_array[user_for_example].reshape(1, -1)  # Asegurar forma correcta para cosine_similarity

# To calculate similarity between the user example embeding and the whole content from filtered data
content_similarities = cosine_similarity(user_embedding_example, all_embeddings_from_filtered_data_array).flatten()

# Sort indexes by similarity
most_similar_indexes = content_similarities.argsort()[::-1]

# Top-10
topten_most_similar_indexes = most_similar_indexes[:10]

In [ ]:
# To display the most similar indexes and their similarity scores
print("Most similar indexes:", most_similar_indexes[:10])
print("Highest similarities:", content_similarities[most_similar_indexes[:10]])

In [ ]:
# search most_similar_indexes, and preferences, and get recommendations
user_id = user_pref.loc[user_for_example]['userId']
movies_preferred = user_pref[user_pref['userId']==user_id]['TitulosPeliculas']
series_preferred = user_pref[user_pref['userId']==user_id]['TitulosSeries']

# Displaying Preferences & Recommendations
print(f'''User {user_id} Preferences:
      ''')
print(f'''Movies preference:''')
for movie in movies_preferred.iloc[0].split(';'):
    print(f'      {movie.strip()}')

print(f'''TV Shows preference:''')
for series in series_preferred.iloc[0].split(';'):
    print(f'      {series.strip()}')

print(f'''Recomendations for user: {user_id}
      ''')

recomendations_user = filtered_data.loc[topten_most_similar_indexes]['CleanTitle']
for recommendation in recomendations_user:
    print(f'      {recommendation}')
